In [2]:
from pybaseball.lahman import *
import numpy as np

teams_df = teams()
salaries_df = salaries()

In [4]:
year = 2016

# Get data from lahman for queried year and 5 years previous
historical_team_data = teams_df.loc[(teams_df['yearID'] >= (year-5))&(teams_df['yearID'] < year)].copy()
historical_team_data['attendancePerGame'] = np.ceil(historical_team_data['attendance']/historical_team_data['Ghome'])
historical_team_data['WSWinBool'] = np.where(historical_team_data['WSWin'] == 'Y', 1, 0)

# Sum in case a team changes ball parks or something such that they have multiple entries
historical_team_data = historical_team_data.groupby('teamID', as_index=False).agg({'attendancePerGame': 'sum', 'WSWinBool': 'sum'})
historical_team_data['avgAttendance'] = np.ceil(historical_team_data['attendancePerGame']/5)

current_team_data = teams_df.loc[(teams_df['yearID'] == year)]

historical_and_current = pd.merge(current_team_data, historical_team_data[['teamID', 'avgAttendance', 'WSWinBool']], on='teamID', how='inner')

# Get num players in top 10% of salaries for current year, add to current year dataframe

salaries_current_year = salaries_df.loc[(salaries_df['yearID']==year)]

num_players = len(salaries_current_year)
ten_percent_amount = int(num_players/10)

top_ten_percent = salaries_current_year.nlargest(ten_percent_amount, 'salary')
top_ten_percent_teams = top_ten_percent.groupby('teamID', as_index=False).agg({'salary': 'count'})

# Combine with historical and current data

all_parameters = pd.merge(historical_and_current, top_ten_percent_teams, on='teamID', how='left')
all_parameters['salary'].fillna(0, inplace=True)

# Get ranking for each individual stat
positive_stats = ['W', 'WSWinBool', 'avgAttendance', 'salary', 'HR', 'SB']

for stat in positive_stats:
    all_parameters[stat + '_rank'] = all_parameters[stat].rank(ascending=0)

all_parameters['total'] = all_parameters['W_rank'] + all_parameters['WSWinBool_rank'] + all_parameters['avgAttendance_rank'] + all_parameters['salary_rank'] + all_parameters['HR_rank'] + all_parameters['SB_rank']
all_parameters['total_rank'] = all_parameters['total'].rank(ascending=1, method='dense')

sortedDf = all_parameters.sort_values('total_rank')
sortedDf

# Send response with data needed in addition to the ranking so front end can display calculation

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,WSWinBool,salary,W_rank,WSWinBool_rank,avgAttendance_rank,salary_rank,HR_rank,SB_rank,total,total_rank
3,2016,AL,BOS,BOS,E,1,162,81.0,93,69,...,1,6.0,5.0,3.0,6.0,2.5,9.0,13.0,38.5,1.0
27,2016,AL,TEX,TEX,W,1,162,81.0,95,67,...,0,6.0,2.5,17.5,7.0,2.5,7.0,10.0,46.5,2.0
9,2016,AL,DET,DET,C,2,161,81.0,86,75,...,0,7.0,12.0,17.5,9.0,1.0,8.0,23.0,70.5,3.0
25,2016,NL,SLN,STL,C,2,162,81.0,86,76,...,1,0.0,12.0,3.0,4.0,23.0,2.0,29.0,73.0,4.0
29,2016,NL,WAS,WSN,E,1,162,81.0,95,67,...,0,0.0,2.5,17.5,14.0,23.0,11.0,6.5,74.5,5.0
28,2016,AL,TOR,TOR,E,2,162,81.0,89,73,...,0,3.0,7.5,17.5,17.0,7.0,4.0,25.0,78.0,6.0
24,2016,NL,SFN,SFG,W,2,162,81.0,87,75,...,2,0.0,9.5,1.0,3.0,23.0,28.0,14.0,78.5,7.0
15,2016,NL,MIL,MIL,C,4,162,81.0,73,89,...,0,1.0,22.0,17.5,11.0,13.5,15.0,1.0,80.0,8.0
2,2016,AL,BAL,BAL,E,2,162,81.0,89,73,...,0,5.0,7.5,17.5,21.0,4.0,1.0,30.0,81.0,9.0
5,2016,NL,CHN,CHC,C,1,162,81.0,103,58,...,0,0.0,1.0,17.5,10.0,23.0,13.0,20.5,85.0,10.0


In [11]:
# Calculate NERD for a team for current year

year=2017
# THIS CODE IS TAKEN FROM THE FOLLOWING SOURCE
# https://github.com/Metlover/NERD-Bot/blob/master/team_nerd_score.py
import requests
from bs4 import BeautifulSoup as bs
import datetime

spotrac_ids = ['Boston Red Sox', 'San Francisco Giants', 'Chicago Cubs',
'Washington Nationals', 'Los Angeles Dodgers',
'Los Angeles Angels of Anaheim', 'New York Yankees', 'Toronto Blue Jays',
'St. Louis Cardinals', 'New York Mets', 'Houston Astros',
'Seattle Mariners', 'Texas Rangers', 'Baltimore Orioles',
'Colorado Rockies', 'Detroit Tigers', 'Cleveland Indians',
'Arizona Diamondbacks', 'Kansas City Royals', 'Minnesota Twins',
'Atlanta Braves', 'Philadelphia Phillies', 'Miami Marlins',
'San Diego Padres', 'Cincinnati Reds', 'Pittsburgh Pirates',
'Milwaukee Brewers', 'Tampa Bay Rays', 'Oakland Athletics',
'Chicago White Sox']

fangraphs_ids = ['Red Sox','Giants','Cubs','Nationals','Dodgers','Angels',
'Yankees','Blue Jays','Cardinals','Mets','Astros','Mariners','Rangers',
'Orioles','Rockies','Tigers','Indians','Diamondbacks','Royals','Twins',
'Braves','Phillies','Marlins','Padres','Reds','Pirates','Brewers','Rays',
'Athletics','White Sox']

team_url = ('https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=c,53,11,6,111,199,3,58&season=%s&month=0&season1=%s&ind=0&team=0,ts&rost=&age=&filter=&players=0' % (str(year-1),str(year-1)))
team_page = requests.get(team_url)
team_soup = bs(team_page.content, 'html.parser')
table = team_soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="LeaderBoard1_dg1_ctl00")
df = pd.read_html(str(table),header=1)
df = df[0]
df = df.iloc[:-1]
bullpen_url = ('https://www.fangraphs.com/leaders.aspx?pos=all&stats=rel&lg=all&qual=0&type=c,6,62&season=%s&month=0&season1=%s&ind=0&team=0,ts&rost=0&age=0&filter=&players=0' % (str(year-1),str(year-1)))
bullpen_page = requests.get(bullpen_url)
bullpen_soup = bs(bullpen_page.content, 'html.parser')
table = bullpen_soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="LeaderBoard1_dg1_ctl00")
bp_df = pd.read_html(str(table),header=1)
bp_df = bp_df[0]
bp_df = bp_df.iloc[:-1]
df = df.merge(bp_df, on='Team')
#This part, a pain in the ass, attaches payroll data from SportTrac to the main DF
payroll_url = 'https://www.spotrac.com/mlb/payroll/' #yo, please don't use this to do previous years because payroll data might be off
payroll_page = requests.get(payroll_url)
payroll_soup = bs(payroll_page.content,'lxml')
table = payroll_soup.find_all('table')[0]
pr_df = pd.read_html(str(table),header=0)
pr_df = pr_df[0]
pr_df = pr_df.loc[:14].append(pr_df.loc[17:]) #removes annoying "league average" stuff
for index in range(0,len(spotrac_ids)): #replaces spotrac ids with FanGraphs IDs
    pr_df['Team'].loc[pr_df['Team'] == spotrac_ids[index]] = fangraphs_ids[index]
df = df.merge(pr_df, on='Team')
#grabs pitching WAR and Wins to determine team luckiness
pitch_url = ('https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,59,4&season=%s&month=0&season1=%s&ind=0&team=0,ts&rost=0&age=0&filter=&players=0' %  (str(year-1),str(year-1)))
pitch_page = requests.get(pitch_url)
pitch_soup = bs(pitch_page.content, 'html.parser')
table = pitch_soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="LeaderBoard1_dg1_ctl00")
pit_df = pd.read_html(str(table),header=1)
pit_df = pit_df[0]
pit_df = pit_df.iloc[:-1]
df = df.merge(pit_df, on='Team')
#grabs park factors for adjusting HR rate
pf_url = ('https://www.fangraphs.com/guts.aspx?type=pf&teamid=0&season=%s' % (str(year-1)))
pf_page = requests.get(pf_url)
pf_soup = bs(pf_page.content,'html.parser')
table = pf_soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="GutsBoard1_dg1_ctl00")
pf_df = pd.read_html(str(table),header=0)
pf_df = pf_df[0]
df = df.merge(pf_df, on='Team')
#calculates NERD
df['zBat'] = (df['Bat'] - df['Bat'].mean())/df['Bat'].std(ddof=0) #Finds Z-score values for calculation
df['HRpPA'] = df['HR_x'] / df['PA'] #calculates HR per PA per team
df['HRpPA'] = df['HRpPA'] * df['HR_y'] / 100 #adjusts for park factors
df['zHRpPA'] = (df['HRpPA'] - df['HRpPA'].mean())/df['HRpPA'].std(ddof=0)
df['zBsR'] = (df['BsR'] - df['BsR'].mean())/df['BsR'].std(ddof=0)
df['zBull'] = -((df['xFIP'] - df['xFIP'].mean())/df['xFIP'].std(ddof=0))
df['zDef'] = (df['Def'] - df['Def'].mean())/df['Def'].std(ddof=0)
df['2019 Total Payroll'] = df['2019 Total Payroll'].replace('[\$,]', '', regex=True).astype(float) #converts dollar amounts to numbers
df['zPay'] = -((df['2019 Total Payroll'] - df['2019 Total Payroll'].mean())/df['2019 Total Payroll'].std(ddof=0))
df['zAge'] = -((df['Age'] - df['Age'].mean())/df['Age'].std(ddof=0))
df['tWAR'] = df['WAR_x'] + df['WAR_y']
df['Luck'] = (df['tWAR'] - df['W'])/20 #Adjusted for calculation
df['zPay'][df['zPay'] < 0] = 0 #replaces values. These throw warnings for no reason because Pandas can't act like R without throwing a goddamn fit
df['zAge'][df['zAge'] < 0] = 0
df['Luck'][df['Luck'] < 0] = 0
df['Luck'][df['Luck'] > 2] = 2
df['NERD'] = df['zBat'] + df['zHRpPA'] + df['zBsR'] + (df['zBull'] / 2) + (df['zDef'] / 2) + df['zPay'] + df['zAge'] + df['Luck'] #unadjusted NERD
df['NERD'] = (((df['NERD'] - min(list(df['NERD']))) * (10)) / (max(list(df['NERD'])) - min(list(df['NERD'])))) #feature scaled
df['NERD'].head()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be s

0     8.765284
1     6.904960
2    10.000000
3     4.752711
4     5.433622
Name: NERD, dtype: float64